# Get started with the Sampler primitive

Learn how to set up and use the Sampler primitive program.


## Overview

The Sampler primitive lets you more accurately contextualize counts. It takes a user circuit as an input and generates an error-mitigated readout of quasiprobabilities. This enables you to more efficiently evaluate the possibility of multiple relevant data points in the context of destructive interference.  


## Prepare the environment

1. Follow the steps in the [getting started guide](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/getting_started.html) to get your Quantum Service instance ready to use.
2. You'll need at least one circuit to submit to the program. Our examples all have circuits in them, but if you want to submit your own circuit, you can use Qiskit to create one. To learn how to create circuits by using Qiskit, see the [Circuit basics tutorial](https://qiskit.org/documentation/tutorials/circuits/01_circuit_basics.html).


## Start a session

With Qiskit Runtime primitives, we introduce the concept of a session or a factory that allows you to define a job as a collection of iterative calls to the quantum computer. When you start a session, it caches the data you send so it doesn't have to be transmitted to the Quantum Datacenter on each iteration.


### Create a Sampler instance

The Sampler class takes in a runtime options dictionary to control the execution environment.

- **options**: Runtime options dictionary that control the execution environment.
    * **backend**: Optional instance of :class:`qiskit_ibm_runtime.IBMBackend` class or
        string name of backend, if not specified a backend will be selected
        automatically (IBM Cloud only).
    * **image**: the runtime image used to execute the program, specified in
        the form of ``image_name:tag``. Not all accounts are
        authorized to select a different image.
    * **log_level**: logging level to set in the execution environment. The valid
        log levels are: ``DEBUG``, ``INFO``, ``WARNING``, ``ERROR``, and ``CRITICAL``.
        The default level is ``WARNING``.

The Sampler class also takes in the following arguments as settings:

- **transpilation_settings**: (EXPERIMENTAL setting, can break between releases without warning)
    Qiskit transpiler settings. The transpilation process converts
    operations in the circuit to those supported by the backend, swaps qubits with the
    circuit to overcome limited qubit connectivity and some optimizations to reduce the
    circuit's gate count where it can.

    * **skip_transpilation**: Transpilation is skipped if set to True.
        False by default.

    * **optimization_settings**:
        * **level**: How much optimization to perform on the circuits.
            Higher levels generate more optimized circuits,
            at the expense of longer transpilation times.
            * 0: no optimization
            * 1: light optimization
            * 2: heavy optimization
            * 3: even heavier optimization
            If ``None``, level 1 will be chosen as default.

- **resilience_settings**: (EXPERIMENTAL setting, can break between releases without warning)
    Using these settings allows you to build resilient algorithms by
    leveraging the state of the art error suppression, mitigation and correction techniques.

    * **level**: How much resilience to build against errors.
        Higher levels generate more accurate results,
        at the expense of longer processing times.
        * 0: no resilience
        * 1: light resilience
        If ``None``, level 0 will be chosen as default.


With these arguments, you can create a Sampler instance with the desired settings. Example:

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit import QuantumCircuit

service = QiskitRuntimeService()

options = { "backend": "ibmq_qasm_simulator" }
transpilation_settings = { "optimization_level": 1 }

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

with Session(service=service) as session:
    sampler = session.sampler(options=options, transpilation_settings=transpilation_settings)

## Write to & read from a session

Running a job and returning the results are done by writing to and reading from the session. The session closes when the code exits the `with` block.

### Specify program inputs

The Sampler.run takes in the following arguments:

- **circuits**: A list of (parameterized) circuits that you want to investigate.
- **parameters**: A list of parameters for the parameterized circuits. It should be omitted if the circuits provided are not parameterized.
- **parameter_values**: An optional list of concrete parameters to be bound.
- **run_options**: Runtime options dictionary that control the execution environment.
    - **shots**: Number of repetitions of each circuit, for sampling.
    - **qubit_lo_freq**: List of default qubit LO frequencies in Hz.
    - **meas_lo_freq**: List of default measurement LO frequencies in Hz.
    - **schedule_los**: Experiment LO configurations, frequencies are given in Hz.
    - **rep_delay**: Delay between programs in seconds. Only supported on certain
        backends (if ``backend.configuration().dynamic_reprate_enabled=True``).
    - **init_qubits**: Whether to reset the qubits to the ground state for each shot.
    - **use_measure_esp**: Whether to use excited state promoted (ESP) readout for measurements
        which are the terminal instruction to a qubit. ESP readout can offer higher fidelity
        than standard measurement sequences.
  
You can find more details in [the Sampler API reference](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.Sampler.html).

Example:

### Run the job & print results

Run the job, specifying your previously defined inputs and options. In this simple example, there is only one circuit and it does not have parameters.

In each Sampler.run() call, you will use `circuits` to specify which circuits to run and, if applicable,  `parameter_values` specifies which parameter to use with the specified circuit.

In [2]:
# executes a Bell circuit
with Session(service=service) as session:
    sampler = session.sampler()
    sampler.options.backend = "ibmq_qasm_simulator"
    sampler.settings.transpilation.optimization_level = 1
    job = sampler.run(circuits=bell)
    print(job.result())

SamplerResult(quasi_dists=[{'11': 0.486328125, '00': 0.513671875}], metadata=[{'header_metadata': {}, 'shots': 1024}])


## Multiple circuit example

In this example, we specify three circuits, but they have no parameters:

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit import QuantumCircuit

service = QiskitRuntimeService()

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# executes three Bell circuits
with Session(service=service) as session:
    sampler = session.sampler()
    sampler.options.backend = "ibmq_qasm_simulator"
    sampler.settings.transpilation.optimization_level = 1
    job = sampler.run(circuits=[bell]*3)
    print(job.result())

SamplerResult(quasi_dists=[{'11': 0.484375, '00': 0.515625}, {'00': 0.5029296875, '11': 0.4970703125}, {'00': 0.4794921875, '11': 0.5205078125}], metadata=[{'header_metadata': {}, 'shots': 1024}, {'header_metadata': {}, 'shots': 1024}, {'header_metadata': {}, 'shots': 1024}])


## Multiple parameterized circuits example

In this example, we run multiple parameterized circuits. When it is run, this line `result = sampler(circuits=[0, 0, 1], parameter_values=[theta1, theta2, theta3])` specifies which parameter to send to each circuit.  

In our example, the parameter labeled `theta1` is sent to the first circuit, `theta2` is sent to the first circuit, and `theta3` is sent to the second circuit.

In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes

service = QiskitRuntimeService()

# parameterized circuit
pqc = RealAmplitudes(num_qubits=2, reps=2)
pqc.measure_all()
pqc2 = RealAmplitudes(num_qubits=2, reps=3)
pqc2.measure_all()

theta1 = [0, 1, 1, 2, 3, 5]
theta2 = [1, 2, 3, 4, 5, 6]
theta3 = [0, 1, 2, 3, 4, 5, 6, 7]
    
with Session(service) as session:
    sampler = session.sampler()
    sampler.options.backend = "ibmq_qasm_simulator"
    sampler.settings.transpilation.optimization_level = 1

    job = sampler.run(circuits=[pqc, pqc, pqc2], parameter_values=[theta1, theta2, theta3])
    print(job.result())

SamplerResult(quasi_dists=[{'01': 0.3828125, '11': 0.4150390625, '00': 0.1171875, '10': 0.0849609375}, {'01': 0.0419921875, '11': 0.3095703125, '00': 0.0751953125, '10': 0.5732421875}, {'01': 0.7080078125, '11': 0.03515625, '10': 0.08203125, '00': 0.1748046875}], metadata=[{'header_metadata': {}, 'shots': 1024}, {'header_metadata': {}, 'shots': 1024}, {'header_metadata': {}, 'shots': 1024}])


### Result

The results align with the parameter - circuit pairs specified previously.  For example, the first result (`{'00': 0.1376953125, '10': 0.095703125, '01': 0.359375, '11': 0.4072265625}`) is the output of the parameter labeled `theta1` being sent to the first circuit.